# Titanic dataset

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
np.set_printoptions(threshold=50, edgeitems=20)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
PATH = 'data/titanic/'
!ls '{PATH}'

gender_submission.csv  models  test.csv  tmp  train.csv


In [4]:
train = pd.read_csv(f'{PATH}train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test = pd.read_csv(f'{PATH}test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
len(train), len(test)

(891, 418)

In [7]:
from IPython.display import HTML

In [8]:
train.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Carter, Mrs. William Ernest (Lucile Polk)",male,NaN,NaN,NaN,1601,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [9]:
skip_flds = ['PassengerId', 'Name', 'Ticket', 'Cabin'] #,'Pclass', 'Sex', 'Embarked']
cat_vars = ['Pclass', 'Sex', 'Embarked']
cont_vars = ['Age', 'SibSp', 'Parch', 'Fare']
dep = 'Survived'

In [10]:
test[dep] = 0

In [11]:
train.drop(columns=skip_flds, inplace=True)
test.drop(columns=skip_flds, inplace=True);    

In [12]:
#train.Sex = train['Sex'] == 'female'
#test.Sex = test['Sex'] == 'female'

In [13]:
train.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [14]:
test.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
Survived      int64
dtype: object

In [15]:
train = train[cat_vars + cont_vars + [dep]].copy()
test = test[cat_vars + cont_vars + [dep]].copy()

In [16]:
for var in cont_vars: 
    train[var] = train[var].astype('float32')
    test[var] = test[var].astype('float32')

In [17]:
for var in cat_vars: 
    train[var] = train[var].astype('category').cat.as_ordered()
    test[var] = test[var].astype('category').cat.as_ordered()

In [18]:
train_cats(train)

In [19]:
train.dtypes

Pclass      category
Sex         category
Embarked    category
Age          float32
SibSp        float32
Parch        float32
Fare         float32
Survived       int64
dtype: object

In [20]:
test.dtypes

Pclass      category
Sex         category
Embarked    category
Age          float32
SibSp        float32
Parch        float32
Fare         float32
Survived       int64
dtype: object

In [21]:
train.describe(include='all')

,Pclass,Sex,Embarked,Age,SibSp,Parch,Fare,Survived
count,891.0,891,889,714.000000,891.000000,891.000000,891.000000,891.000000
unique,3.0,2,3,NaN,NaN,NaN,NaN,NaN
top,3.0,male,S,NaN,NaN,NaN,NaN,NaN
freq,491.0,577,644,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,29.699118,0.523008,0.381594,32.204208,0.383838
std,NaN,NaN,NaN,14.526497,1.102744,0.806057,49.693428,0.486592
min,NaN,NaN,NaN,0.420000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,20.125000,0.000000,0.000000,7.910400,0.000000
50%,NaN,NaN,NaN,28.000000,0.000000,0.000000,14.454200,0.000000
75%,NaN,NaN,NaN,38.000000,1.000000,0.000000,31.000000,1.000000


In [22]:
df, y, nas, mapper = proc_df(train, dep, do_scale=True)

In [23]:
df.dtypes

Pclass         int8
Sex            int8
Embarked       int8
Age         float64
SibSp       float64
Parch       float64
Fare        float64
Age_na      float64
dtype: object

In [24]:
df.describe(include='all')

,Pclass,Sex,Embarked,Age,SibSp,Parch,Fare,Age_na
count,891.000000,891.000000,891.000000,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02
mean,2.308642,1.647587,2.529742,5.608004e-08,-2.063752e-08,3.043784e-08,-9.674623e-09,-1.644775e-17
std,0.836071,0.477990,0.800254,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00
min,1.000000,1.000000,0.000000,-2.224156e+00,-4.745452e-01,-4.736736e-01,-6.484216e-01,-4.978947e-01
25%,2.000000,1.000000,2.000000,-5.657364e-01,-4.745452e-01,-4.736736e-01,-4.891482e-01,-4.978947e-01
50%,3.000000,2.000000,3.000000,-1.046373e-01,-4.745452e-01,-4.736736e-01,-3.573909e-01,-4.978947e-01
75%,3.000000,2.000000,3.000000,4.333116e-01,4.327933e-01,-4.736736e-01,-2.424636e-02,-4.978947e-01
max,3.000000,2.000000,3.000000,3.891555e+00,6.784163e+00,6.974147e+00,9.667167e+00,2.008457e+00


In [25]:
test.head()

,Pclass,Sex,Embarked,Age,SibSp,Parch,Fare,Survived
0,3,male,Q,34.5,0.0,0.0,7.8292,0
1,3,female,S,47.0,1.0,0.0,7.0000,0
2,2,male,Q,62.0,0.0,0.0,9.6875,0
3,3,male,S,27.0,0.0,0.0,8.6625,0
4,3,female,S,22.0,1.0,1.0,12.2875,0


In [26]:
df_test, _, nas, mapper = proc_df(test, dep, do_scale=True, na_dict=nas, mapper=mapper)
nas

{'Age': 28.0, 'Fare': 14.4542}

In [27]:
df_test.describe(include='all')

,Pclass,Sex,Embarked,Age,SibSp,Parch,Fare,Age_na
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,1.636364,2.401914,0.034078,-0.068631,0.013345,0.067901,0.017766
std,0.841838,0.481622,0.854496,0.973531,0.813665,1.218251,1.124524,1.014388
min,1.000000,1.000000,1.000000,-2.243368,-0.474545,-0.473674,-0.648422,-0.497895
25%,1.000000,1.000000,2.000000,-0.488887,-0.474545,-0.473674,-0.489442,-0.497895
50%,3.000000,2.000000,3.000000,-0.104637,-0.474545,-0.473674,-0.357391,-0.497895
75%,3.000000,2.000000,3.000000,0.490949,0.432793,-0.473674,-0.014745,-0.497895
max,3.000000,2.000000,3.000000,3.584155,6.784163,10.698057,9.667167,2.008457


In [28]:
len(df)

891

In [29]:
val_idx = get_cv_idxs(len(df), val_pct=0.2)
len(val_idx)

178

In [30]:
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..., 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0])

In [31]:
model = ColumnarModelData.from_data_frame(PATH, val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=16, test_df=df_test)

In [32]:
cat_sizes = [(cat, len(train[cat].cat.categories)) for cat in cat_vars]
cat_sizes

[('Pclass', 3), ('Sex', 2), ('Embarked', 3)]

In [33]:
embedding_sizes = [(size, min(50, (size+1)//2)) for _,size in cat_sizes]
embedding_sizes

[(3, 2), (2, 1), (3, 2)]

In [34]:
learner = model.get_learner(embedding_sizes, len(df.columns)-len(cat_vars),
                           0.04, 1, [100, 100], [0.01,0.1], y_range=(0,1))

In [35]:
#learner.crit = nn.BCEWithLogitsLoss()

In [36]:
learner.lr_find(start_lr=1e-5,end_lr=2)

  0%|          | 0/45 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THC/THCTensorCopy.cu:204

In [ ]:
learner.sched.plot()

In [ ]:
learner.fit(0.001, 4, cycle_len=2, cycle_mult=2)